### How to perform data collection


1. First, 

# Capture a Video

In [2]:
from tkinter import Tk, Label, Entry, Button, Canvas
import cv2
from PIL import Image, ImageTk
import threading
import numpy as np
import os
import time






def prepare_directory(player_id):
    base_folder = "player_videos"
    player_folder = os.path.join(base_folder, player_id)
    if not os.path.exists(player_folder):
        os.makedirs(player_folder)
    start_time = time.strftime("%Y%m%d-%H%M%S")
    session_folder = os.path.join(player_folder, start_time)
    os.makedirs(session_folder)
    return session_folder

def crop_and_resize_frame(frame, size=(256, 256)):
    height, width = frame.shape[:2]
    min_dim = min(height, width)
    top = (height - min_dim) // 2
    left = (width - min_dim) // 2
    cropped_frame = frame[top:top+min_dim, left:left+min_dim]
    resized_frame = cv2.resize(cropped_frame, size, interpolation=cv2.INTER_AREA)
    return resized_frame

class VideoCapture:
    def __init__(self, frame_rate=10, batch_size=100, player_id="", gui_update_callback=None):
        self.frame_rate = frame_rate
        self.batch_size = batch_size
        self.player_id = player_id
        self.is_capturing = False
        self.frames = []
        self.batch_count = 0
        self.folder_name = ""
        self.gui_update_callback = gui_update_callback
        self.capture_thread = None

    def start_capture(self):
        if self.player_id:
            self.folder_name = prepare_directory(self.player_id)
            self.is_capturing = True
            self.capture_thread = threading.Thread(target=self.capture_and_save_batches)
            self.capture_thread.start()

    def stop_capture(self):
        self.is_capturing = False
        if self.capture_thread:
            self.capture_thread.join(timeout=1)
            if self.capture_thread.is_alive():
                print("Capture thread did not terminate. Attempting to force stop.")
            else:
                print("Capture stopped successfully.")
        if self.frames:
            # Ensure the last batch is saved when capturing stops
            batch_time_stamp = time.strftime("%Y%m%d-%H%M%S")
            batch_array = np.array(self.frames)
            np.savez_compressed(os.path.join(self.folder_name, f"batch_{batch_time_stamp}_{self.batch_count}"), batch_array)
        self.frames = []

    def capture_and_save_batches(self):
        cap = cv2.VideoCapture(0)
        try:
            while self.is_capturing:
                ret, frame = cap.read()
                if ret and self.is_capturing:  # Check if capturing should continue
                    modified_frame = crop_and_resize_frame(frame)
                    if self.gui_update_callback:
                        self.gui_update_callback(modified_frame)
                    self.frames.append(modified_frame)
                    if len(self.frames) == self.batch_size:
                        batch_time_stamp = time.strftime("%Y%m%d-%H%M%S")
                        batch_array = np.array(self.frames)
                        np.savez_compressed(os.path.join(self.folder_name, f"batch_{batch_time_stamp}_{self.batch_count}"), batch_array)
                        self.frames = []  # Clear frames for the next batch
                        self.batch_count += 1
                else:
                    break
        finally:
            cap.release()


class WebCam_Application(Tk):
    def __init__(self):
        super().__init__()
        self.capture = None
        self.title("Video Capture")
        self.geometry("300x400")
        Label(self, text="Player ID:").pack()
        self.player_id_entry = Entry(self)
        self.player_id_entry.pack()
        self.start_button = Button(self, text="Start Capture", command=self.start_capture)
        self.start_button.pack()
        self.stop_button = Button(self, text="Stop Capture", command=self.stop_capture, state="disabled")
        self.stop_button.pack()
        self.canvas = Canvas(self, width=256, height=256)
        self.canvas.pack()

    def update_gui_with_frame(self, frame):
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        im = Image.fromarray(frame)
        imgtk = ImageTk.PhotoImage(image=im)
        self.canvas.create_image(128, 128, image=imgtk)
        self.canvas.imgtk = imgtk  # Keep a reference to the image to prevent garbage collection, whatever that shit is

    def start_capture(self):
        player_id = self.player_id_entry.get()
        if player_id:
            self.capture = VideoCapture(player_id=player_id, gui_update_callback=self.update_gui_with_frame)
            self.capture.start_capture()
            self.start_button.config(state="disabled")
            self.stop_button.config(state="normal")

    def stop_capture(self):
        if self.capture:
            self.capture.stop_capture()
            self.start_button.config(state="normal")
            self.stop_button.config(state="disabled")



In [6]:
from tkinter_app_webcam import *

In [7]:
app = WebCam_Application()
app.mainloop()

#### How many frames were in a certain video?

In [2]:
import numpy as np
import cv2
import os

def load_frames_from_npz(player_id, session_start):
    """
    Load all frames from NPZ files for a given player ID and session start.

    Parameters:
    - player_id: The ID of the player.
    - session_start: The start time of the session to uniquely identify the session folder.

    Returns:
    - A list of all frames from the session.
    """
    base_dir = "player_videos"
    session_dir = os.path.join(base_dir, str(player_id), session_start)
    frames = []
    for npz_file in sorted(os.listdir(session_dir)):
        if npz_file.endswith('.npz'):
            file_path = os.path.join(session_dir, npz_file)
            data = np.load(file_path, allow_pickle=True)
            batch_frames = [frame for _, frame in data.items()][0]
            frames.extend(batch_frames)
    return frames


def display_frame(frame, frame_number):
    """
    Display a single frame with OpenCV.

    Parameters:
    - frame: The frame to display.
    - frame_number: The frame number (for window title).
    """
    window_name = f"Frame {frame_number}"
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    cv2.imshow(window_name, frame)
    cv2.waitKey(0)  # Wait until any key is pressed
    cv2.destroyAllWindows()

In [4]:
player_id = "0"

session_start = "20240223-123525"  # Format: YearMonthDay-HourMinuteSecond
frames = load_frames_from_npz(player_id, session_start)
print(f"Total frames loaded: {len(frames)}")


Total frames loaded: 154


### View a single frame

- Can use this to calibrate the video by recording a stop watch to make sure we can allign the sensor data to this video data
- But to get around this i have also given each batch a timestamp
- This video footage is not 10 fps but rather 28 or so

In [5]:

# Display an early frame and a later frame
early_frame_number = 0  # rmeber to press q to quit these windowsss
later_frame_number = len(frames) - 1  

display_frame(frames[early_frame_number], early_frame_number)
display_frame(frames[later_frame_number], later_frame_number)
